<div style="
    display: block;
    padding: 12px 20px;
    background-color: #1A73E8;
    color: white;
    border-radius: 30px;
    font-family: 'Helvetica Neue', Arial, sans-serif;
    font-size: 16px;
    font-weight: 600;
    margin: 15px auto;
    width: fit-content;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    text-align: center;
    letter-spacing: 0.5px;
">
    <strong>Translate with Various AI Models</strong>
</div>

<div style="
    display: block;
    padding: 12px 20px;
    background-color: #66BB6A;
    color: white;
    border-radius: 30px;
    font-family: 'Helvetica Neue', Arial, sans-serif;
    font-size: 16px;
    font-weight: 600;
    margin: 15px auto;
    width: fit-content;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    text-align: center;
    letter-spacing: 0.5px;
">
    <strong>Simon-Pierre Boucher</strong>
</div>

<div style="
    display: block;
    padding: 12px 20px;
    background-color: #FFA726;
    color: white;
    border-radius: 30px;
    font-family: 'Helvetica Neue', Arial, sans-serif;
    font-size: 16px;
    font-weight: 600;
    margin: 15px auto;
    width: fit-content;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    text-align: center;
    letter-spacing: 0.5px;
">
    <strong>2024-09-14</strong>
</div>


This Python script translates a given text from one language to another using multiple models provided by OpenAI, Anthropic, and Mistral APIs. Here's a detailed explanation of its components:

### 1. **Environment Setup**:
   - **`load_dotenv()`**: Loads environment variables from a `.env` file, including API keys for OpenAI, Anthropic, and Mistral, enabling secure access to their respective APIs.

### 2. **Translation Functions**:
   - **`openai_translate_text()`**:
     - This function sends a translation request to OpenAI’s models (e.g., `gpt-4`).
     - The prompt specifies the source and target languages and includes the text to be translated.
     - It returns the translated text or an error message if the request fails.

   - **`anthropic_translate_text()`**:
     - Similar to the OpenAI function, this sends a request to the Anthropic API (e.g., `claude-3-5-sonnet`) to translate the provided text from one language to another.
     - The function formats the request and returns the translation generated by the model.

   - **`run_mistral()`**:
     - This is a helper function that sends user input (in this case, the translation request) to the Mistral API for processing.
     - It manages the API request and returns the translation generated by the model.

   - **`mistral_translate_text()`**:
     - This function builds the translation task for the Mistral API and calls `run_mistral()` to generate the translation.

### 3. **Aggregated Translation**:
   - **`translate_text_with_all_models()`**:
     - This function iterates over lists of models from OpenAI, Anthropic, and Mistral, sending translation requests for the same text.
     - It stores the translation results from each model in a dictionary, with the model names as keys and the translated text as values.
     - The function processes multiple models from each API and gathers their outputs.

### 4. **Main Program Execution**:
   - **API Keys and Input Text**:
     - The API keys are loaded from the environment variables.
     - The text to be translated (about polar bears and climate change) is provided, along with the source language (`English`) and target language (`French`).

   - **Model Lists**:
     - Lists of models for OpenAI (`gpt-3.5-turbo`, `gpt-4`), Anthropic (`claude-3-5-sonnet`, `claude-3-opus`), and Mistral (`open-mistral-7b`, `mistral-medium-latest`) are defined for translation evaluation.

   - **Generating Translations**:
     - The function `translate_text_with_all_models()` is called to generate translations from all the models for the same input text.
   
   - **Results Output**:
     - The script prints the translated results from each model, including the model name and word count of the translated text.

### Purpose:
This script is useful for comparing how different AI models from OpenAI, Anthropic, and Mistral handle translation tasks. By running the same translation task across multiple models, it allows users to evaluate and benchmark the quality and performance of each model for translation from English to French.

In [1]:
import os
from dotenv import load_dotenv
import requests
import json

# Charger les variables d'environnement
load_dotenv()

/Users/simon-pierreboucher/Desktop/notebook/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [2]:
def openai_translate_text(api_key, text, source_lang, target_lang, model="gpt-4", temperature=0.7, max_tokens=1024, stop=None):
    """
    Translates a given text from source language to target language using the OpenAI API.
    """
    task_description = f"Translate the following text from {source_lang} to {target_lang}."

    prompt_content = f"""
    {task_description}

    Text: {text}
    Translation:
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    if stop:
        data["stop"] = stop

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        translated_text = response_json["choices"][0]["message"]["content"].strip()
        return translated_text
    else:
        return f"Error {response.status_code}: {response.text}"

In [3]:
def anthropic_translate_text(api_key, text, source_language, target_language, model="claude-3-5-sonnet-20240620", max_tokens=1024, temperature=0.7):
    """
    Translates a given text from a source language to a target language using the Anthropic API.
    """
    url = "https://api.anthropic.com/v1/messages"
    
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    data = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {"role": "user", "content": f"Please translate the following text from {source_language} to {target_language}:\n\n{text}"}
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        translated_text = response_json["content"][0]["text"].strip()
        return translated_text
    else:
        return f"Error {response.status_code}: {response.text}"



In [4]:
def run_mistral(api_key, user_message, model="mistral-medium-latest"):
    url = "https://api.mistral.ai/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": user_message}
        ],
        "temperature": 0.7,
        "top_p": 1.0,
        "max_tokens": 512,
        "stream": False,
        "safe_prompt": False,
        "random_seed": 1337
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        return response_json["choices"][0]["message"]["content"].strip()
    else:
        return f"Error {response.status_code}: {response.text}"

In [5]:
def mistral_translate_text(api_key, text, source_lang, target_lang, model="mistral-medium-latest"):
    """
    Translates a given text from source language to target language using the Mistral API.
    """
    user_message = f"Translate the following text from {source_lang} to {target_lang}:\n\n{text}"
    return run_mistral(api_key, user_message, model=model)

In [6]:
def translate_text_with_all_models(openai_key, anthropic_key, mistral_key, text, source_lang, target_lang, openai_models, anthropic_models, mistral_models, temperature=0.7, max_tokens=100, stop=None):
    results = {}

    # Traduire du texte avec tous les modèles OpenAI
    for model in openai_models:
        openai_result = openai_translate_text(openai_key, text, source_lang, target_lang, model, temperature, max_tokens, stop)
        results[f'openai_{model}'] = openai_result

    # Traduire du texte avec tous les modèles Anthropic
    for model in anthropic_models:
        anthropic_result = anthropic_translate_text(anthropic_key, text, source_lang, target_lang, model, max_tokens, temperature)
        results[f'anthropic_{model}'] = anthropic_result

    # Traduire du texte avec tous les modèles Mistral
    for model in mistral_models:
        mistral_result = mistral_translate_text(mistral_key, text, source_lang, target_lang, model)
        results[f'mistral_{model}'] = mistral_result

    return results

In [7]:
	
if __name__ == "__main__":
    openai_key = os.getenv("OPENAI_API_KEY")
    anthropic_key = os.getenv("ANTHROPIC_API_KEY")
    mistral_key = os.getenv("MISTRAL_API_KEY")
    text_to_translate = "Polar bears are increasingly threatened by climate change. As the Arctic ice melts, their habitat shrinks, making it difficult for them to hunt seals, their primary food source."
    source_lang = "English"
    target_lang = "French"

    openai_models = ["gpt-3.5-turbo", "gpt-4", "gpt-4-turbo", "gpt-4o-mini", "gpt-4o"]
    anthropic_models = ["claude-3-5-sonnet-20240620", "claude-3-opus-20240229", "claude-3-sonnet-20240229", "claude-3-haiku-20240307"]
    mistral_models = ["open-mistral-7b", "open-mixtral-8x7b", "open-mixtral-8x22b", "mistral-small-latest", "mistral-medium-latest", "mistral-large-latest"]

    results = translate_text_with_all_models(openai_key, anthropic_key, mistral_key, text_to_translate, source_lang, target_lang, openai_models, anthropic_models, mistral_models)
    
    for model_name, result in results.items():
        word_count = len(result.split())
        print(f"\033[1mResult from {model_name} ({word_count} words):\033[0m\n{result}\n")

Result from openai_gpt-3.5-turbo (38 words):
Les ours polaires sont de plus en plus menacés par le changement climatique. Alors que la glace de l'Arctique fond, leur habitat diminue, ce qui rend difficile pour eux de chasser les phoques, leur principale source de nourriture.

Result from openai_gpt-4 (37 words):
Les ours polaires sont de plus en plus menacés par le changement climatique. Avec la fonte de la glace arctique, leur habitat se réduit, rendant difficile pour eux la chasse aux phoques, leur principale source de nourriture.

Result from openai_gpt-4-turbo (37 words):
Les ours polaires sont de plus en plus menacés par le changement climatique. À mesure que la glace arctique fond, leur habitat se réduit, rendant difficile pour eux de chasser les phoques, leur principale source de nourriture.

Result from openai_gpt-4o-mini (35 words):
Les ours polaires sont de plus en plus menacés par le changement climatique. À mesure que la glace arctique fond, leur habitat se réduit, rendant 